In [ ]:



import numpy as np
from copy import deepcopy


def totalPrice(indChromosone,weights,values, weight):
  if sum(indChromosone * weights) > weight:
    return 100000
  return -sum(indChromosone * values)




class problemTreasureTrove:
  def __init__(self):
    self.number_of_genes = 1000  # i.e. number of items in treasure
    self.weight = 60     
    self.weights = np.random.uniform(1,10,self.number_of_genes)
    self.values = np.random.randint(0,50,self.number_of_genes)
    self.cost_function = lambda chromo:totalPrice(chromo,self.weights,self.values,self.weight)    # This allows the problem class to pass in other parameters to cost function


class individual:
  # This class defines the individual for the genetic algorithm. We hope to find the individual which solves our problem
  chromosone = None

  def __init__(self, prob):    #  This is the constructor for the individual and as such needs the problem to mould the individuals to it
    #Create a random individual.  (Too random produces entire population of "useless individuals")
    # self.chromosone =np.random.choice([True,False],prob.number_of_genes) 

    # If instead we pick a "few" items 
    self.chromosone =np.random.choice([True,False],prob.number_of_genes, p = [0.01,0.99]) 
    self.cost = prob.cost_function(self.chromosone)
    

  def crossover(self,other_parent, exploreRate = 0):
    cutPoint = np.random.randint(len(self.chromosone))
    child1 = deepcopy(self)
    child2 = deepcopy(other_parent)
    child1.chromosone = np.append(self.chromosone[:cutPoint], other_parent.chromosone[cutPoint:])  #first part of parent 1 second part parent 2
    child2.chromosone = np.append(other_parent.chromosone[:cutPoint], self.chromosone[cutPoint:])   # vice versa
    return child1,child2

  def mutate(self,mutation_rate, mutation_range = 0):
    for index in range (len(self.chromosone)):
      if np.random.uniform()<mutation_rate:
        self.chromosone[index] = not self.chromosone[index]


class parameters:
  def __init__(self):
    self.number_in_population = 100
    self.number_of_generations = 10
    self.child_rate = 1
    self.crossover_explore = 0.1
    self.mutation_rate = 0.01
    self.range_of_gene_mutation = 0.3

def choose_different_indices(max_value):
  index1 = np.random.randint(0,max_value)
  index2 = np.random.randint(0,max_value)
  if index1 == index2:
    return choose_different_indices(max_value)
  return index1, index2

def run_genetic(prob, params):
  #  read the problem
  cost_function = prob.cost_function

  #   read parameters
  number_in_population = params.number_in_population
  max_number_of_iterations = params.number_of_generations
  number_of_children = params.child_rate * number_in_population
  explore_rate_crossover = params.crossover_explore
  mutation_rate = params.mutation_rate 
  range_of_mutation = params.range_of_gene_mutation

  #  Initialise the population
  best_solution = individual(prob)
  best_solution.cost = 999999

  population = []
  for i in range(number_in_population):
    new_individual = individual(prob)
    population.append(new_individual)
    if new_individual.cost < best_solution.cost:
      best_solution = deepcopy(new_individual)  # copy new_individual


  # loop over the generations (or till solution is found) 
  for iteration in range(max_number_of_iterations):
    #generate children
    children = []
    while len(children) < number_of_children:
      # choose parents
      parent1_index, parent2_index = choose_different_indices(len(population))
      parent1 = population[parent1_index]
      parent2 = population[parent2_index]

      child1, child2 = parent1.crossover(parent2, explore_rate_crossover)

      #Mutate Children
      child1.mutate(mutation_rate, range_of_mutation)
      child2.mutate(mutation_rate, range_of_mutation)

      #  cost the children
      child1.cost = cost_function(child1.chromosone)
      child2.cost = cost_function(child2.chromosone)

      children.append(child1)
      children.append(child2)

    # add children to popuation
    population += children

    # sort and cull population

    population = sorted(population, key = lambda x: x.cost)
    population = population[:number_in_population]

    if population[0].cost < best_solution.cost:
      best_solution = deepcopy(population[0])

    for i in range(0,len(population),10): 
      print("Generation {}:   Population Position {} : Soution Cost {}".format(iteration,i, population[i].cost))

  return population, best_solution


In [ ]:
probTT = problemTreasureTrove()
paramsTT = parameters()

In [ ]:
i1 = individual(probTT)
i2 = individual(probTT)

In [ ]:
pop,bs = run_genetic(probTT,paramsTT)